# Data Preparation
## Step 1 : Data  Sources

In our analysis we look at data  on movie releases from IMDB, and The Numbers to answer questions for a hypothetical movie studio start-up. Our data are contained in the ../zippedData directory of this repo and will need to be unzipped and imported to be useful for this analysis. 

### Why did we choose our data?

We decided to use data from `tn.movie_budgets.csv.gz`, `imdb.title.basics.csv.gz`,`imdb.title.crew.csv.gz` and `imdb.name.basics.csv.gz`. We chose `tn.movie_budgets.csv.gz` because it provided more detailed information about revenue and production costs which allowed us to ask and answer more meaningful questions about the overall return on investment for each film. We also included `imdb.title.basics.csv.gz` in order to take a more detailed look at what _types_ of films performed best over time. Finally, we took a look at the personell files in `imdb.name.basics.csv.gz` and `imdb.title.crew.csv.gz` to answer questions about which directors were involved in successful titles.

First we will import the required packages and build an unzip function to help access our relevant files.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import gzip
from io import StringIO

%matplotlib inline

In [2]:
# This function uses gzip and io to unzip the data, two modules which were initially added to our repo by Jesus
def unzip_csv(file_location): 
    file = gzip.open(file_location, 'rb')
    content = file.read()
    file.close()
    try:
        content_str = str(content,'utf-8')
        content_data = StringIO(content_str) 
    except:
        content_str = str(content,'latin-1')
        content_data = StringIO(content_str) 
    try:
        return pd.read_csv(content_data)
    except:
        return pd.read_csv(content_data, sep='\t')

In [3]:
!ls -la ../zippedData/

total 23100
drwxr-xr-x 1 smang 197609        0 Jun 22 19:51 .
drwxr-xr-x 1 smang 197609        0 Jun 22 22:09 ..
-rw-r--r-- 1 smang 197609    53544 Jun 22 19:51 bom.movie_gross.csv.gz
-rw-r--r-- 1 smang 197609 18070960 Jun 22 19:51 imdb.name.basics.csv.gz
-rw-r--r-- 1 smang 197609  3459897 Jun 22 19:51 imdb.title.basics.csv.gz
-rw-r--r-- 1 smang 197609  1898523 Jun 22 19:51 imdb.title.crew.csv.gz
-rw-r--r-- 1 smang 197609   153218 Jun 22 19:51 tn.movie_budgets.csv.gz


In [4]:
#hard-coding the file-locations and nicknames into a dict for future reference
file_locations = ['../zippedData/imdb.name.basics.csv.gz'
                  ,'../zippedData/imdb.title.basics.csv.gz'
                  ,'../zippedData/tn.movie_budgets.csv.gz'
                  ,'../zippedData/imdb.title.crew.csv.gz']

file_nicknames = ['name','basics','budgets','crew']

#this dicitonary comprehension uses a zip function to smush the two lists together and then parse them into a dict
#we also have a reference for each raw df and its location on the drive.
file_dict = {k:v for k,v in zip(file_nicknames,file_locations)}

#we unzip and define frames
name= unzip_csv(file_dict['name'])
basics= unzip_csv(file_dict['basics'])
budgets= unzip_csv(file_dict['budgets'])
crew= unzip_csv(file_dict['crew'])

data_list = [
    name
    ,crew
    ,basics
    ,budgets
]

## Step 2: Cleaning the Data

In the next step we take the raw data frames and format the values to their appropriate data types, drop duplicates, null values, and redundant or irrelevant columns. We'll examine the head and info of each data frame as a starting off point:

In [5]:
for df in data_list:
    display(df.head(2))

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"


,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"


,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"


In [6]:
#lets iterate through each data frame in the list and gather some intel about the data
for df in data_list:
    display(df.head(2),df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   nconst              606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   float64
 3   death_year          6783 non-null    float64
 4   primary_profession  555308 non-null  object 
 5   known_for_titles    576444 non-null  object 
dtypes: float64(2), object(4)
memory usage: 27.8+ MB


,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   tconst     146144 non-null  object
 1   directors  140417 non-null  object
 2   writers    110261 non-null  object
dtypes: object(3)
memory usage: 3.3+ MB


,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"


...

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"


None

>It looks like we have missing values in 3 of our 4 dataframes so we will have to do something about that with our data preparation. We also have a lot less information on the budgets than we do on any other file so we will preserve those records as priority. We can't make a very in depth analysis with that dataframe alone. We also want information about
_type_ of movie we're looking at, so we're going to need to match up the files by title to add genre information into the picture. Finally, we're going to need to use the crew and name data to associate the correct director to the correct movie.

Lets create a set of functions and loops that will do some of the basic cleaning for us right out of the gate. These functions will handle tasks that we want done to every dataframe in our possesion. Firstly we will want it to strip lurking whitespace out of all the object columns and column headers. Then we will look at a lambda function that drops null values for a specified set of columns so we can preserve only records that will help our analysis. We're going about it this way because the budgets df doesnt have any null values and the other three are so much more massive than that one dropping null values wont damage our pool of records very much in the end. 

In [7]:
def strip(df):
    df.columns = df.columns.str.strip()
    for column in df.columns:
        try:
            df.column = df.column.str.strip
        except:
            pass

#We'll need to get a list of the series we want formatted as money from budgets before hand.
#since its the only one with int columns we'll only develop the function this far for now.

def clean_money(df_series):
    #the map function applys the .replace to each cell in the given series, x[1:] skips the $
    return df_series.map(lambda x: int(x[1:].replace(',','')))

def dropna(dropna_dict):
    for df in dropna_dict.keys():
        print(df.values())

In [8]:
for df in data_list:
    df = strip(df)

In [9]:
#laundering the money
budgets['production_budget'] = clean_money(budgets['production_budget'])
budgets['domestic_gross'] = clean_money(budgets['domestic_gross'])
budgets['worldwide_gross'] = clean_money(budgets['worldwide_gross'])

In [10]:
#We'll only drop null values in the columns that are important for our merging the columns together. 
#A graphic representation of the relationships is provided in repo as key_scheme.png in the images folder

name.drop(
    name[name.nconst.isna()==True].index
    ,inplace=True
)

crew.drop(
    crew[crew.tconst.isna()==True].index
    ,inplace=True
)


crew.drop(
    crew[crew.directors.isna()==True].index
    ,inplace=True
)

basics.drop(
    basics[basics.tconst.isna()==True].index
    ,inplace=True
)

basics.drop(
    basics[basics.genres.isna()==True].index
    ,inplace=True
)

#We're going to skip the movie column becuase we're going to merge the release year to the end of each movie title
#later to differntiate duplicate movie titles by year

for df in data_list:
    display(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 606648 entries, 0 to 606647
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   nconst              606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   float64
 3   death_year          6783 non-null    float64
 4   primary_profession  555308 non-null  object 
 5   known_for_titles    576444 non-null  object 
dtypes: float64(2), object(4)
memory usage: 32.4+ MB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140417 entries, 0 to 146142
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   tconst     140417 non-null  object
 1   directors  140417 non-null  object
 2   writers    109008 non-null  object
dtypes: object(3)
memory usage: 4.3+ MB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140736 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           140736 non-null  object 
 1   primary_title    140736 non-null  object 
 2   original_title   140734 non-null  object 
 3   start_year       140736 non-null  int64  
 4   runtime_minutes  112233 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 7.5+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   int64 
 4   domestic_gross     5782 non-null   int64 
 5   worldwide_gross    5782 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 271.2+ KB


None

In [11]:
#id column is a redundant index so we're dropping it
budgets.drop('id', axis=1, inplace=True)

#setting date column to datatime object for use in charts etc.
budgets['release_date'] = pd.to_datetime(budgets['release_date'])

#stripping any unseen or unknown whitespace from the object locales
budgets.columns.str.strip()
budgets['movie'] = budgets['movie'].str.strip()

#this function launders the money ;D
def clean_money(budgets_series):
    #the map function applys the .replace to each cell in the given series, x[1:] skips the $
    return budgets_series.map(lambda x: int(x[1:].replace(',','')))

budgets['production_budget'] = clean_money(budgets['production_budget'])
budgets['domestic_gross'] = clean_money(budgets['domestic_gross'])
budgets['worldwide_gross'] = clean_money(budgets['worldwide_gross'])

#adding in relevant columns
budgets['foreign_gross'] = budgets.worldwide_gross - budgets.domestic_gross
budgets['profit'] = budgets.worldwide_gross - budgets.production_budget

#dropping duplicates
budgets.drop_duplicates('movie', keep='first',inplace=True)

TypeError: 'int' object is not subscriptable

In [ ]:
#looks good now
budgets.info()

### Basics
Now the general shape of the cleaning process has been defined we can rinse and repeat on our other data sets, making them easier to use in later analysis.

In [ ]:
basics.head()

In [ ]:
basics.info()

In [ ]:
basics['movie'] = basics['primary_title']

#keeping only 'movie' and 'ttconst' as keys for our other data, and 'genres' for further analysis
basics.drop(['primary_title','original_title','runtime_minutes'],axis=1,inplace=True)

The columns look correct:

In [ ]:
basics.columns

In [ ]:
#the strip functions remove unwanted whitespace if its lurking in there
basics.columns = basics.columns.str.strip()

for column in list(basics.columns):
    try:
        basics[column] = basics[column].str.strip()
    except:
        pass

#Dropping duplicates
basics.drop_duplicates('movie', keep='first', inplace=True)

#Dropping null vales
to_drop = basics[basics['genres'].isna()==True].index
basics.drop(to_drop,inplace=True)

#this .map will apply a .split to all the genres at each "," decoding the genres data into a nested list.
#basics['genres'] = basics['genres'].map(lambda x: x.split(","))

As demonstrated below the previously difficult to use string data has now been munged into a useful format:

In [ ]:
basics['genres']

In [ ]:
basics['genres'][0][0]

### Name

In [ ]:
name.head()

In [ ]:
name.info()

In [ ]:
name.isnull().sum()

In [ ]:
#dropping these since they're outside the scope of our analysis
name.drop(['primary_profession','birth_year','death_year','known_for_titles'],axis=1,inplace=True)

#cleaning the object data
name.columns = name.columns.str.strip()

#for loop will work here since all columns are object data
for column in list(name.columns):
    name[column] = name[column].str.strip()

In [ ]:
name.head(5)

### Crew

In [ ]:
crew.head()

In [ ]:
crew.info()

In [ ]:
#dropping these since they're outside the scope of our analysis
crew.drop(['writers'],axis=1,inplace=True)

#cleaning the object data
crew.columns = crew.columns.str.strip()

#for loop will work here since all columns are object data
for column in list(crew.columns):
    crew[column] = crew[column].str.strip()

Since the director is the only reason we're using this data set we're going to drop null director values.

In [ ]:
to_drop = crew[crew.directors.isna()==True].index
crew.drop(to_drop,inplace=True)

In [ ]:
crew.info()

In [ ]:
crew.directors = crew.directors.map(lambda x: x.split(","))

In [ ]:
crew.head()

In [ ]:
files_list = [name,basics,crew,budgets]

In [ ]:
for file in files_list:
    display(file.head())

In [ ]:
budgets['year'] = pd.DatetimeIndex(budgets['release_date']).year

In [ ]:
budgets['movie'] = budgets['movie'] +' '+ budgets['year'].astype(str)

In [ ]:
budgets['movie'].head()

In [ ]:
basics['movie'] = basics['movie'] +' '+ basics['start_year'].astype(str)

In [ ]:
df = budgets.merge(basics)
df.head()

In [ ]:
df.info()

In [ ]:
name.head()

In [ ]:
crew.head()

In [ ]:
df = df.merge(crew,how='left')
df.info()

In [ ]:
df_e = df.explode('directors')

In [ ]:
df_e = df_e.merge(name,left_on='directors',right_on='nconst')
df_e.columns

In [ ]:
df_e.drop(['directors','nconst','tconst'],axis=1,inplace=True)

In [ ]:
df_e.head()
df_e['primary_name'].isna().sum()

In [ ]:
df_e = df_e.groupby('movie')['primary_name'].apply(", ".join).reset_index()

In [ ]:
df = df.merge(df_e)

In [ ]:
df['director'] = df['primary_name']
df.drop(['tconst','directors','primary_name'],axis=1,inplace=True)
df.head()

In [ ]:
df['genres'].loc[0][0]

In [ ]:
df.info()

In [ ]:
df['year'] = df['start_year']
df.drop(['release_date','start_year'],axis=1,inplace=True)

In [ ]:
!pwd

In [ ]:
df.to_csv('data-clean.csv')
df.to_excel('data-clean.xlsx')